## Problem 36
The decimal number,  585=10010010012
  (binary), is palindromic in both bases.

Find the sum of all numbers, less than one million, which are palindromic in base  10
  and base  2
 .

(Please note that the palindromic number, in either base, may not include leading zeros.)

In [1]:
import numpy as np

In [2]:
def check_palindrome(x):
    if type(x) != str:
        x = str(x)
    if x == x[::-1]:
        return True
    return False

def check_palindrome_base2(x):
    base2 = np.base_repr(x, 2)
    return check_palindrome(base2)

def check(x):
    if check_palindrome(x):
        if check_palindrome_base2(x):
            return True
    return False


In [3]:
palindromes = [x for x in range(1000000) if check(x)]

In [4]:
sum(palindromes)

872187

In [5]:
#the palindrome-first approach
def create_base2_strings(last_string):
    if len(last_string) > 20:
        return []
    else:
        # Add a one:
        next_is_one = last_string + "1"
        futures_with_one = create_base2_strings(next_is_one)

        # Add a zero:
        next_is_zero = last_string + "0"
        futures_with_zero = create_base2_strings(next_is_zero)

        # Combine the results
        return [last_string] + futures_with_one + futures_with_zero

# Start with an empty string
result = create_base2_strings("")

In [6]:
#drop the ones that end with 0
base_strings = [r for r in result if not r.endswith("0") and len(r) > 0]

In [7]:
#all palindromes in the list and the entire set up to len 10 mirrored on itself
base2_palindromes = [x for x in base_strings if check_palindrome(x)] + [x[::-1]+x for x in base_strings if len(x) <= 10]
base2_palindromes = np.unique(base2_palindromes)

In [8]:
dec10 = [int(b, 2) for b in base2_palindromes]

In [9]:
dec10_palindromes = [d for d in dec10 if check_palindrome(d)]

In [10]:
sum(dec10_palindromes)

872187

## Problem 37 - Truncatable Primes

<p>The number $3797$ has an interesting property. Being prime itself, it is possible to continuously remove digits from left to right, and remain prime at each stage: $3797$, $797$, $97$, and $7$. Similarly we can work from right to left: $3797$, $379$, $37$, and $3$.</p>
<p>Find the sum of the only eleven primes that are both truncatable from left to right and right to left.</p>
<p class="smaller">NOTE: $2$, $3$, $5$, and $7$ are not considered to be truncatable primes.</p>


In [11]:
import numpy as np
from sympy import isprime

In [12]:
def is_truncatable(input_string):
    for i in range(1, len(input_string)):
        if not isprime(int(input_string[:i])) or not isprime(int(input_string[i:])):
            return False
    return True

In [13]:
def recursively_make_primes(input_string):
    #if our input string is not prime, we return an empty list
    #becasue this line of enquiry is exhausted
    if not isprime(int(input_string)):
        return []

    #If it is prime, we add it to our list of valid results
    results = [input_string]

    #then we can loop over digits to check appending and prepending options for
    for addition in ["1", "3", "5", "7", "9"]:

        #check the line of appending the digit
        if not addition == 5: #appending 5 wont ever result in a prime
            in_front = addition + input_string
            
            #Go down the line of this append, which terminates when appending isnt prime
            from_in_front_base = recursively_make_primes(in_front)
            results.extend(from_in_front_base)
            
        #check the line of prepending the digit which terminates when prepending isnt prime
        behind = input_string + addition
        from_behind_base = recursively_make_primes(behind)
        results.extend(from_behind_base)
    return results

In [14]:
#find the primes
primes = []
for digit in [str(x) for x in range(1, 10)]:
    primes.extend(recursively_make_primes(digit))

In [15]:
#Get the uniques, because we are finding primes multiple times from different starting points
primes = [p for p in np.unique(primes) if int(p) > 10]

In [16]:
truncatable_primes = [int(p) for p in primes if is_truncatable(p)]
truncatable_primes.sort()

In [17]:
print(f"We have found {len(truncatable_primes)} truncatable primes: {truncatable_primes}")

We have found 11 truncatable primes: [23, 37, 53, 73, 313, 317, 373, 797, 3137, 3797, 739397]


In [18]:
sum(truncatable_primes)

748317

## Problem 38 - Pandigital Multiples

<p>Take the number $192$ and multiply it by each of $1$, $2$, and $3$:</p>
\begin{align}
192 \times 1 = 192\\
192 \times 2 = 384\\
192 \times 3 = 576
\end{align}
<p>By concatenating each product we get the $1$ to $9$ pandigital, $192384576$. We will call $192384576$ the concatenated product of $192$ and $(1,2,3)$.</p>
<p>The same can be achieved by starting with $9$ and multiplying by $1$, $2$, $3$, $4$, and $5$, giving the pandigital, $918273645$, which is the concatenated product of $9$ and $(1,2,3,4,5)$.</p>
<p>What is the largest $1$ to $9$ pandigital $9$-digit number that can be formed as the concatenated product of an integer with $(1,2, \dots, n)$ where $n \gt 1$?</p>


---
So I guess we can infer from the question that the number we are looking for is higher than 918_273_645, and cant be higher than 999_999_999. An added constraint is that there can be no zeros or repeating digits. That reduces the search space considerably. 

I guess our first number, which will result from the multipication with 1, has to be equal to or larger than the digits found in our lower bound, and we just have to check to the next power of 10 of that digit? So we check 9, 91...98, (99 has repeating digits), 918, 919, 921..987 etc. That also reduces the search space if we start from this angle. There is also a constraint to the max starting candidate, set by the max amount of digits being 9. We cant look for larger than 4 digit numbers because combining the first two options would result in a 10 digit string which is not going to be pandigital. I think that means the max for our candidates is the largest option under 5 digits, 9876. 

This is beginning to feel like a basic search: 

Lets say for an integer x, we concatenate x with x*n for n = 2,3,4 etc. As soon as we get repeating digits or a zero we abandon x as a valid option, and check the next valid candidate for x. Lets try to put this into some code.

In [19]:
#we need a function to check if a number is pandigital
def is_pandigital(x: int) -> bool:
    x = str(x)
    if "0" not in x and len(np.unique([*x])) == 9:
        return True
    return False

In [20]:
#we need a function to check for doubles and zeros in potential but incomplete pandigitals
def has_zeros_or_doubles(x: int) -> bool:
    x = str(x)
    if "0" in x:
        return True
    else:
        for digit in x:
            if x.count(digit) > 1:
                return True
    return False

In [21]:
#find out if a pandigital can be made by subsequently adding the product and checking conditions
def find_pandigital(x: int):
    stringx = str(x)
    n = 2
    while not is_pandigital(stringx):   
        new_stringx = stringx + str(x*n)
        if has_zeros_or_doubles(new_stringx):
            return
        else:
            n+=1
            stringx = new_stringx
    return int(stringx)

In [22]:
%%time
#our potential candidates for x are all numbers larger than prefixes from the given pandigital in the excercise, and smaller than
candidates = [x for x in range(92, 98)] + [x for x in range(921, 9876) if not has_zeros_or_doubles(x)]

#find potential pandigitals
pds = [find_pandigital(x) for x in candidates]

#filter out none vals
pds = [p for p in pds if p!=None]

#print the largest found pandigital
print(f"The largest found pandigital is {max(pds)}")

The largest found pandigital is 932718654
CPU times: user 12.9 ms, sys: 10.9 ms, total: 23.7 ms
Wall time: 22.9 ms


## Problem 39 - Integer Right Triangles

<p>If $p$ is the perimeter of a right angle triangle with integral length sides, $\{a, b, c\}$, there are exactly three solutions for $p = 120$.</p>
<p>$\{20,48,52\}$, $\{24,45,51\}$, $\{30,40,50\}$</p>
<p>For which value of $p \le 1000$, is the number of solutions maximised?</p>


In [23]:
def make_triplet(n):
    return ((n^2)-1, (n^2) +1, 2*n)

In [49]:
def find_max_p():
    solutions = []
    for a in range(1, 1000):
        for b in range(a, 1000):
            c = np.sqrt(a**2 + b**2)
            if c>b and int(c) == c and a+b+c < 1000:
                solutions.append(int(a+b+c))
    return solutions

In [50]:
solutions = find_max_p()

In [51]:
counts = {s: solutions.count(s) for s in np.unique(solutions)}
    

In [52]:
maxp = max(counts, key = counts.get)

In [53]:
maxp

840

In [54]:
counts[840]

8

## Problem 40 - Champernowne's Constant

<p>An irrational decimal fraction is created by concatenating the positive integers:
$$0.12345678910{\color{red}\mathbf 1}112131415161718192021\cdots$$</p>
<p>It can be seen that the $12$<sup>th</sup> digit of the fractional part is $1$.</p>
<p>If $d_n$ represents the $n$<sup>th</sup> digit of the fractional part, find the value of the following expression.
$$d_1 \times d_{10} \times d_{100} \times d_{1000} \times d_{10000} \times d_{100000} \times d_{1000000}$$</p>

In [29]:
#abuse modern hardware, build the million digit string of champernowes constant:
n = 1
champs_const = ""
while(len(champs_const)) < 1e6:
    champs_const += str(n)
    n+=1
    

In [30]:
#find the digits at the right indexes:
digits = [int(champs_const[n-1]) for n in [1, 10, 100, 1000, 10000, 100000, 1000000]]

In [31]:
#get the product of the found digits:
np.product(digits)

210